::::
:::{thebe-button}
:::
::::

# Fit tracks

In [ ]:
from __future__ import annotations

from boilercore.fits import Fit, fit_from_params
from boilercv_dev.docs.nbs import init
from boilercv_pipeline.models.path import get_datetime
from boilercv_pipeline.stages import get_thermal_data
from boilercv_pipeline.stages.find_tracks import FindTracks as Params
from devtools import pprint
from more_itertools import one
from numpy import clip, inf, logspace, nan_to_num, pi
from pandas import concat, read_hdf

from boilercv.dimensionless_params import jakob, prandtl

In [ ]:
params = Params(context=init(), only_sample=True)
params.set_display_options()
C = params.cols

thermal = read_hdf(params.deps.thermal)
TC = get_thermal_data.Cols()

all_tracks = list(one(params.dfs).parent.iterdir())
times = [get_datetime(p.stem) for p in all_tracks]

# Physical parameters
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3

# Fit constants
GUESSES = logspace(-2, 0)

pprint(params)

In [ ]:
tracks = (
    concat(
        read_hdf(p, key="dst").assign(**{TC.time(): time})
        for p, time in zip(all_tracks, times, strict=True)
    )
    .set_index(TC.time())
    .assign(**{
        TC.subcool(): thermal.set_index(TC.time()).loc[times, TC.subcool()],  # pyright: ignore[reportArgumentType, reportCallIssue]
        "jakob": lambda df: jakob(
            liquid_density=LIQUID_DENSITY,
            vapor_density=VAPOR_DENSITY,
            liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
            subcooling=df[TC.subcool()],
            latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
        ),
    })
    .reset_index()
)
Pr = prandtl(
    dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
    isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
)
Ja = tracks["jakob"].median()
Re_b0 = tracks[C.bub_reynolds0()].median()

In [ ]:
C_2_lucic_mayinger_2010 = 0.61
C_3_lucic_mayinger_2010 = 0.33


def nusselt_lucic_mayinger_2010(Re_b, C_1, C_4):  # noqa: D103, N803
    return C_1 * Re_b**C_2_lucic_mayinger_2010 * Pr**C_3_lucic_mayinger_2010 * Ja**C_4


fits = {"C_1": inf, "C_4": inf}
errors = {"C_1_err": inf, "C_4_err": inf}
for guess in GUESSES:
    new_fits, new_errors = fit_from_params(
        model=nusselt_lucic_mayinger_2010,
        params=Fit(
            independent_params=(["Re_b"]),
            free_params=(["C_1", "C_4"]),
            values={"C_1": guess, "C_4": guess},
            bounds={"C_1": (0, inf), "C_4": (0, inf)},
        ),
        x=tracks[C.bub_reynolds()].values,
        y=tracks[C.bub_nusselt()].values,
    )
    if new_errors["C_1_err"] < errors["C_1_err"]:
        fits = new_fits
        errors = new_errors
        display({**errors, **fits})

display({
    "C_2": C_2_lucic_mayinger_2010,
    "C_3": C_3_lucic_mayinger_2010,
    "C_5": 2 * fits["C_1"] * (2 - C_2_lucic_mayinger_2010),
    "C_6": 1 + fits["C_4"],
    "C_7": 1 / (2 - C_2_lucic_mayinger_2010),
})

In [ ]:
C_2_chen_mayinger_1992 = 0.7
C_3_chen_mayinger_1992 = 0.5


def nusselt_chen_mayinger_1992(Re_b, C_1, C_4):  # noqa: D103, N803
    return C_1 * Re_b**C_2_chen_mayinger_1992 * Pr**C_3_chen_mayinger_1992 * Ja**C_4


fits = {"C_1": inf, "C_4": inf}
errors = {"C_1_err": inf, "C_4_err": inf}
for guess in GUESSES:
    new_fits, new_errors = fit_from_params(
        model=nusselt_chen_mayinger_1992,
        params=Fit(
            independent_params=(["Re_b"]),
            free_params=(["C_1", "C_4"]),
            values={"C_1": guess, "C_4": guess},
            bounds={"C_1": (0, inf), "C_4": (0, inf)},
        ),
        x=tracks[C.bub_reynolds()].values,
        y=tracks[C.bub_nusselt()].values,
    )
    if new_errors["C_1_err"] < errors["C_1_err"]:
        fits = new_fits
        errors = new_errors
        display({**errors, **fits})

display({
    "C_2": C_2_chen_mayinger_1992,
    "C_3": C_3_chen_mayinger_1992,
    "C_5": 2 * fits["C_1"] * (2 - C_2_chen_mayinger_1992),
    "C_6": 1 + fits["C_4"],
    "C_7": 1 / (2 - C_2_chen_mayinger_1992),
})

In [ ]:
def beta_florschuetz_chao_1965(Fo_0, C_1):  # noqa: N803, D103
    return 1 - 4 * Ja * (Fo_0 / pi) ** C_1


fits, errors = fit_from_params(
    model=beta_florschuetz_chao_1965,
    params=Fit(
        independent_params=(["Fo_0"]),
        free_params=(["C_1"]),
        values={"C_1": 1},
        bounds={"C_1": (0, inf)},
    ),
    x=tracks[C.bub_fourier()].values,
    y=tracks[C.bub_beta()].values,
)
display({**errors, **fits})

In [ ]:
C_6_chen_mayinger_1992 = 1.00
C_7_chen_mayinger_1992 = 1 / (2 - C_2_chen_mayinger_1992)


def beta_chen_mayinger_1992(x, C_5):  # noqa: N803, D103
    Fo_0, Ja = x.T  # noqa: N806
    return clip(
        nan_to_num(
            (
                1
                - C_5
                * Fo_0
                * Ja**C_6_chen_mayinger_1992
                * Pr**C_2_chen_mayinger_1992
                * Re_b0**C_3_chen_mayinger_1992
            )
            ** (1 / (2 - C_2_chen_mayinger_1992)),
            nan=0,
        ),
        0,
        None,
    )


fits, errors = fit_from_params(
    model=beta_chen_mayinger_1992,
    params=Fit(
        independent_params=(["x"]),
        free_params=(["C_5"]),
        values={"C_5": 1},
        bounds={"C_5": (0, inf)},
    ),
    x=tracks[[C.bub_fourier(), "jakob"]].values,
    y=tracks[C.bub_beta()].values,
)
display({
    **errors,
    **fits,
    "C_1": fits["C_5"] / 2 / (2 - C_2_chen_mayinger_1992),
    "C_2": C_2_chen_mayinger_1992,
    "C_3": C_3_chen_mayinger_1992,
    "C_4": C_6_chen_mayinger_1992 - 1,
    "C_6": C_6_chen_mayinger_1992,
    "C_7": C_7_chen_mayinger_1992,
})